In [1]:
# -*- coding: utf-8 -*-
# Import the required libraires
import numpy as np
import pandas as pd




test_neg = pd.read_csv('./data/test_Arabic_tweets_negative.tsv',header=None, sep='\t')
test_pos = pd.read_csv('./data/test_Arabic_tweets_positive.tsv',header=None, sep='\t')
train_neg = pd.read_csv('./data/train_Arabic_tweets_negative.tsv',header=None, sep='\t')
train_pos = pd.read_csv('./data/train_Arabic_tweets_positive.tsv',header=None, sep='\t')

data = pd.concat([test_neg, test_pos, train_neg, train_pos])
data = data.drop([0],axis=1)
data

,1
0,حتى الايتونز خربتوه مو صاحين انتو؟؟ 😭
1,واحد تبع النظام السوري يقول أن المخابرات السور...
2,الى متى التعامل السئ للخادمات وعدم احترامهم وك...
3,رايح جاي ي طحلبي 🐸 #الهلال_الاهلي
4,تتمغط ومعها سداع 😫
...,...
22756,السحب الليلة على الايفون .. رتويت للمرفقة وطبق...
22757,😂 لابسة احمر ليه يا ست انتي ايه المناسبة 😂
22758,كلاام جمييل تستاهل(من احبه الله جعل محبته ف قل...
22759,- ألطف صورة ممكن تعبر عن رمضان 💙


In [2]:
import re
!pip install camel_tools
from camel_tools.tokenizers.word import simple_word_tokenize
from nltk.stem import WordNetLemmatizer
!pip install arabic-stopwords
import arabicstopwords.arabicstopwords as stp
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
  
stopwords = set(stp.stopwords_list())
def clean_review(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                            َ    | # Fatha
                            ً    | # Tanwin Fath
                            ُ    | # Damma
                            ٌ    | # Tanwin Damm
                            ِ    | # Kasra
                            ٍ    | # Tanwin Kasr
                            ْ    | # Sukun
                            ـ     # Tatwil/Kashida
                        """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
    text = simple_word_tokenize(text)
    text = " ".join(WordNetLemmatizer().lemmatize(i) for i in text if i not in stopwords)

    return text

  Using cached camel_tools-1.2.0-py3-none-any.whl
  Using cached transformers-4.22.2-py3-none-any.whl (4.9 MB)
  Using cached editdistance-0.6.0.tar.gz (29 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached huggingface_hub-0.10.0-py3-none-any.whl (163 kB)
  Running setup.py clean for editdistance
Failed to build editdistance
  Running setup.py install for editdistance: started
  Running setup.py install for editdistance: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [12 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\editdistance
  copying editdistance\__init__.py -> build\lib.win-amd64-cpython-310\editdistance
  copying editdistance\_editdistance.h -> build\lib.win-amd64-cpython-310\editdistance
  copying editdistance\def.h -> build\lib.win-amd64-cpython-310\editdistance
  running build_ext
  building 'editdistance.bycython' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for editdistance
  error: subprocess-exited-with-error
  
  Running setup.py in

ModuleNotFoundError: No module named 'camel_tools'

In [4]:
data[1] = data[1].apply(lambda x:clean_review(x))
data

,1
0,حتي الايتونز خربتوه مو صاحين انتو ؟ ؟ 😭
1,واحد تبع النظام السوري يقول ان المخابرات السور...
2,الي متي التعامل السء للخادمات وعدم احترامهم وك...
3,رايح جاي طحلبي 🐸 # الهلال _ الاهلي
4,تتمغط سداع 😫
...,...
7801,يشاء الله ، يستبدل اسبابا باسباب ، ، يشاء الله...
7802,# اكتفي ✋🏼 كفاره المجلس سبحانگ اللهہم وبحمدگ ا...
7803,✨ اللهم اني اسالك لطف القدر وحلاوه الايام وسعا...
7804,# _ قلبك _ غرد رااحه قلبك تكمن . . سماع # القر...


In [5]:

# from camel_tools.disambig.mle import MLEDisambiguator
# from camel_tools.tagger.default import DefaultTagger

# from camel_tools.ner import NERecognizer

# ner = NERecognizer.pretrained()
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from textblob import TextBlob

data['POS'] = data[1].apply(lambda x: TextBlob(x).tags)
data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


,1,POS
0,حتي الايتونز خربتوه مو صاحين انتو ؟ ؟ 😭,"[(حتي, JJ), (الايتونز, NNP), (خربتوه, NNP), (م..."
1,واحد تبع النظام السوري يقول ان المخابرات السور...,"[(واحد, JJ), (تبع, NNP), (النظام, NNP), (السور..."
2,الي متي التعامل السء للخادمات وعدم احترامهم وك...,"[(الي, JJ), (متي, NNP), (التعامل, NNP), (السء,..."
3,رايح جاي طحلبي 🐸 # الهلال _ الاهلي,"[(رايح, JJ), (جاي, NNP), (طحلبي, NNP), (🐸, NNP..."
4,تتمغط سداع 😫,"[(تتمغط, JJ), (سداع, NNP), (😫, NN)]"
...,...,...
7801,يشاء الله ، يستبدل اسبابا باسباب ، ، يشاء الله...,"[(يشاء, JJ), (الله, NNP), (،, NNP), (يستبدل, N..."
7802,# اكتفي ✋🏼 كفاره المجلس سبحانگ اللهہم وبحمدگ ا...,"[(اكتفي, NNP), (✋🏼, NNP), (كفاره, NNP), (المجل..."
7803,✨ اللهم اني اسالك لطف القدر وحلاوه الايام وسعا...,"[(✨, JJ), (اللهم, NNP), (اني, NNP), (اسالك, NN..."
7804,# _ قلبك _ غرد رااحه قلبك تكمن . . سماع # القر...,"[(_, NNP), (قلبك, NNP), (_, NNP), (غرد, NNP), ..."


In [6]:
# !pip install camel_tools
!camel_data -i ner-arabert


The following packages will be installed: 'ner-arabert'
Extracting package 'ner-arabert': 100% 542M/542M [00:01<00:00, 290MB/s]


In [7]:
from camel_tools.ner import NERecognizer

ner = NERecognizer.pretrained()

data['NER'] = data[1].apply(lambda x: list(zip(x.split(),ner.predict_sentence(x.split()))))

data

,1,POS,NER
0,حتي الايتونز خربتوه مو صاحين انتو ؟ ؟ 😭,"[(حتي, JJ), (الايتونز, NNP), (خربتوه, NNP), (م...","[(حتي, O), (الايتونز, O), (خربتوه, O), (مو, O)..."
1,واحد تبع النظام السوري يقول ان المخابرات السور...,"[(واحد, JJ), (تبع, NNP), (النظام, NNP), (السور...","[(واحد, O), (تبع, O), (النظام, O), (السوري, O)..."
2,الي متي التعامل السء للخادمات وعدم احترامهم وك...,"[(الي, JJ), (متي, NNP), (التعامل, NNP), (السء,...","[(الي, O), (متي, O), (التعامل, O), (السء, O), ..."
3,رايح جاي طحلبي 🐸 # الهلال _ الاهلي,"[(رايح, JJ), (جاي, NNP), (طحلبي, NNP), (🐸, NNP...","[(رايح, B-MISC), (جاي, I-MISC), (طحلبي, B-ORG)..."
4,تتمغط سداع 😫,"[(تتمغط, JJ), (سداع, NNP), (😫, NN)]","[(تتمغط, O), (سداع, O), (😫, O)]"
...,...,...,...
7801,يشاء الله ، يستبدل اسبابا باسباب ، ، يشاء الله...,"[(يشاء, JJ), (الله, NNP), (،, NNP), (يستبدل, N...","[(يشاء, O), (الله, B-MISC), (،, O), (يستبدل, O..."
7802,# اكتفي ✋🏼 كفاره المجلس سبحانگ اللهہم وبحمدگ ا...,"[(اكتفي, NNP), (✋🏼, NNP), (كفاره, NNP), (المجل...","[(#, O), (اكتفي, O), (✋🏼, O), (كفاره, O), (الم..."
7803,✨ اللهم اني اسالك لطف القدر وحلاوه الايام وسعا...,"[(✨, JJ), (اللهم, NNP), (اني, NNP), (اسالك, NN...","[(✨, O), (اللهم, O), (اني, O), (اسالك, O), (لط..."
7804,# _ قلبك _ غرد رااحه قلبك تكمن . . سماع # القر...,"[(_, NNP), (قلبك, NNP), (_, NNP), (غرد, NNP), ...","[(#, O), (_, O), (قلبك, O), (_, O), (غرد, O), ..."


In [9]:
pos_dict = {}
for rev in data['POS']:
    for word in rev:
        if word[1] in pos_dict:
            #if word[0] not in word[1]:
            pos_dict[word[1]].add(word[0])
        else:
            pos_dict[word[1]] = set(word[0])

In [11]:
for key in pos_dict:
    pos_dict[key] = list(pos_dict[key])

In [13]:
pos_out = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in pos_dict.items() ]))
pos_out

,JJ,NNP,NN,RB,CD,VBP,PRP,VBN,RP,POS,...,VBZ,CC,FW,DT,UH,WP,PRP$,WRB,TO,JJS
0,لله,تبكيگ,لله,فتيحي,۰,اتحب,me,t,o,',...,لله,الرابحين,ه,a,مووجوود,t,your,o,o,s
1,ابقيته,ولامزال,حياء,اصدقاء,25,🇸🇦,I,l,t,NaN,...,✈️,&,فان,the,؟,h,t,h,t,t
2,وعلاش,للكيزاان,كاش,really,20k,🙊,it,f,u,NaN,...,تتعاطف,neg,ل,all,YES,a,h,w,to,r
3,مافيش,كاش,🎈,😴,ذخر,ابناء,he,e,away,NaN,...,دوستويفسكي,تكذب,التايم,another,♥️,what,حرام,NaN,To,e
4,مجاش,بناءا,🙃,neg,⓵,يقتصر,You,Based,NaN,NaN,...,ابتسم,نتمني,يا,A,🖤,chat,i,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43252,NaN,والمفردات,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43253,NaN,ترا,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43254,NaN,مالكوا,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43255,NaN,اريا,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
!pip install xlwt
pos_out.to_excel('postagging.xlsx')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
ner_dict = {}
for rev in data['NER']:
    for word in rev:
        if word[1] in ner_dict:
            #if word[0] not in word[1]:
            ner_dict[word[1]].add(word[0])
        else:
            ner_dict[word[1]] = set(word[0])



In [27]:
for key in ner_dict:
    ner_dict[key] = list(ner_dict[key])

In [28]:
ner_out = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in ner_dict.items() ]))
ner_out

,O,B-MISC,I-MISC,B-ORG,B-PERS,I-PERS,B-LOC,I-ORG,I-LOC
0,تبكيگ,لله,سءال,نتفليكس,عبدالف,لله,سكواد,ه,ياسين
1,ولامزال,سكواد,العوار,فتيحي,غازي,ل,بجده,فوريفر,العربيه
2,للكيزاان,كاش,ستور,تومبويز,جابر,الزهراء,الاحساء,العربيه,القمر
3,كاش,فان,جابر,وللاتحاد,فان,زوما,جزيره,سكواد,العربي
4,بناءا,التايم,كاش,ل,لريم,موسي,اوروجواي,العامه,شروان
...,...,...,...,...,...,...,...,...,...
46861,والمفردات,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46862,ترا,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46863,مالكوا,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46864,اريا,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
!pip install xlwt
ner_out.to_excel('ner.xlsx')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
!camel_data -i morphology-db-all

The following packages will be installed: 'morphology-db-lev-01', 'morphology-db-egy-r13', 'morphology-db-glf-01', 'morphology-db-msa-r13'
Extracting package 'morphology-db-lev-01': 100% 10.6M/10.6M [00:00<00:00, 324MB/s]
Extracting package 'morphology-db-egy-r13': 100% 67.3M/67.3M [00:00<00:00, 248MB/s]
Extracting package 'morphology-db-glf-01': 100% 7.98M/7.98M [00:00<00:00, 438MB/s]
Extracting package 'morphology-db-msa-r13': 100% 40.5M/40.5M [00:00<00:00, 443MB/s]


In [35]:
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger


mled = MLEDisambiguator.pretrained()
tagger = DefaultTagger(mled, 'pos')

tagger.tag('ذهبت الى المدرسة'.split())

FileNotFoundError: ignored